In [1]:
from API_call import *
from functions import get_gamePks,read_gamePks
from datetime import datetime as dt
import os,re,csv
from os import walk
import statsapi as mlb
import pandas as pd
from database import *
from sqlalchemy.exc import IntegrityError

In [2]:
get_gamePks([str(x) for x in range(2008,2020)])   

gamePks=read_gamePks()

In [3]:
db = MyDatabase('sqlite',dbname='mlb.db')

Engine(sqlite://///Users/schlinkertc/code/MLB/mlb.db)


In [4]:
Base.metadata.create_all(db.db_engine)
Base.metadata.bind=db.db_engine

In [5]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db.db_engine)
session = Session()

In [6]:
def insert_game(gamePk):
    gamePk = int(gamePk)
    q = db.db_engine.execute('select pk from games').fetchall()
    existing_pks = [x[0] for x in q]
    
    if gamePk in existing_pks:
        return {'game':gamePk,'insert_status':'fail','reason':'already exists'}
    else:
        try:
            call = API_call(gamePk)
        except:
            return {'game':gamePk,'insert_status':'fail','reason':'API call failed'}

        mappers = [
            Game,Play,
            Pitch,
            Action,Runner,Credit,
            Matchup,Venue,
            Team,GameTeamLink,TeamRecord,
            Player,PitchData,HitData
        ]

        for mapper in mappers:
            insert_values = []

            table = mapper.__table__
            cols = [x.name for x in table.c]

            if mapper == HitData or mapper == PitchData:
                records = call.__dict__['pitches']
            else:
                records = call.__dict__[table.name]
            for record in records:
                insert_value = {}
                for k in cols:
                    try:
                        insert_value[k]=record[k]
                    except KeyError:
                        insert_value[k]=None
                insert_values.append(insert_value)

            conn = db.db_engine.connect()    
            try:
                conn.execute(table.insert(),insert_values)
                return {'game':gamePk,'insert_status':'success','reason':'None'}
            except IntegrityError:
                return {'game':gamePk,'insert_status':'fail','reason':'Integrity Error'}


In [7]:
#[insert_game(game) for game in test_games]

In [8]:
#TEST: which columns, if any, don't have any values
dfs = {}
for table in db.db_engine.table_names():
    df = pd.read_sql(f"select * from {table}",db.db_engine)
    dfs[table]=df


for k in dfs.keys():
    df = dfs[k]
    for col in df.columns:
        if df[col].isna().sum()==df.shape[0]:
            print(k,col)

In [9]:
# pks = [x[0] for x in session.query(Game.pk).all()]

# calls = [API_call(pk) for pk in pks]

# dfs_fromApi = {}
# for k in calls[0].__dict__.keys():
#     if k[0] != '_':
#         records = [x for call in calls for x in call.__dict__[k]]
#         df = pd.DataFrame.from_records(records)
#         dfs_fromApi[k] = df

In [10]:
dfs['pitch_data']

,gamePk,atBatIndex,playEndTime,index,pitchData_startSpeed,pitchData_endSpeed,pitchData_strikeZoneTop,pitchData_zone,pitchData_typeConfidence,pitchData_plateTime,...,pitchData_coordinates_vZ0,pitchData_coordinates_x,pitchData_coordinates_y,pitchData_coordinates_x0,pitchData_coordinates_y0,pitchData_coordinates_z0,pitchData_coordinates_aX,pitchData_breaks_breakY,pitchData_breaks_spinRate,pitchData_breaks_spinDirection
0,566083,0,2019-03-20T09:36:53.000Z,3,NaN,NaN,3.359,NaN,NaN,NaN,...,NaN,144.71,127.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,566083,0,2019-03-20T09:36:53.000Z,4,NaN,NaN,3.359,NaN,NaN,NaN,...,NaN,91.67,130.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,566083,1,2019-03-20T09:38:55.000Z,0,NaN,NaN,3.359,NaN,NaN,NaN,...,NaN,127.17,135.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,566083,1,2019-03-20T09:38:55.000Z,1,NaN,NaN,3.359,NaN,NaN,NaN,...,NaN,124.98,161.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,566083,1,2019-03-20T09:38:55.000Z,2,NaN,NaN,3.359,NaN,NaN,NaN,...,NaN,120.16,117.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779246,599377,78,2019-10-31T03:50:11.000Z,2,86.7,80.7,3.590,14.0,NaN,0.43,...,-2.98,68.62,180.07,-2.40,50.0,5.50,-2.22,24.0,2598.0,186.0
779247,599377,78,2019-10-31T03:50:11.000Z,3,96.0,87.5,3.350,14.0,NaN,0.40,...,-6.90,88.25,183.52,-2.29,50.0,5.33,-12.79,24.0,2597.0,208.0
779248,599377,78,2019-10-31T03:50:11.000Z,4,96.5,87.9,3.530,14.0,NaN,0.40,...,-8.23,52.89,202.28,-2.25,50.0,5.16,-11.49,24.0,2637.0,205.0
779249,599377,78,2019-10-31T03:50:11.000Z,5,95.9,87.2,3.350,7.0,NaN,0.40,...,-7.10,135.02,186.87,-2.52,50.0,5.28,-8.94,24.0,2571.0,200.0


In [31]:
ins = [insert_game(game) for game in gamePks['2019']]
df = pd.DataFrame.from_records(ins)
df

,game,insert_status,reason
0,566083,fail,already exists
1,566958,fail,already exists
2,565603,fail,already exists
3,566564,fail,already exists
4,567055,fail,already exists
...,...,...,...
2611,599373,fail,already exists
2612,599374,fail,already exists
2613,599375,fail,already exists
2614,599376,fail,already exists


In [77]:
call = API_call(567055)

In [78]:
liveData = call._result['liveData']

In [79]:
box_score = liveData['boxscore']

In [80]:
away = box_score['teams']['away']

In [88]:
players = away['players']

In [105]:
player_dicts = []
for k in players.keys():
    player_dicts.append(flatten_dicts(players[k]))



In [106]:
df = pd.DataFrame.from_records(player_dicts)

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Columns: 199 entries, jerseyNumber to stats_pitching_blownSaves
dtypes: bool(4), float64(79), int64(72), object(44)
memory usage: 146.8+ KB


In [116]:
df['status_description'].value_counts()

Signed    46
Active    30
NRI       20
Name: status_description, dtype: int64

In [109]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 199 columns):
jerseyNumber                                   object
parentTeamId                                   float64
person_id                                      int64
person_fullName                                object
person_link                                    object
position_code                                  object
position_name                                  object
position_type                                  object
position_abbreviation                          object
status_code                                    object
status_description                             object
gameStatus_isCurrentBatter                     bool
gameStatus_isCurrentPitcher                    bool
gameStatus_isOnBench                           bool
gameStatus_isSubstitute                        bool
seasonStats_batting_gamesPlayed                int64
seasonStats_batting_flyOuts              